In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Load data

In [2]:
import numpy as np
import polars as pl
import pandas as pd
import os
import gc
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import numpy as np
import polars as pl
import pandas as pd
import os
from tqdm import tqdm

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import coo_matrix, hstack, csr_matrix, vstack
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline

In [4]:
MODEL_DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/Nhom_4/2. Thực Hành/Visualize Data/Model Data'
PRE_DIR = '/content/drive/MyDrive/Colab Notebooks/Nhom_4/2. Thực Hành/Visualize Data/Preprocessed Data'

In [5]:
pl.read_ndjson('/content/drive/MyDrive/Colab Notebooks/Nhom_4/2. Thực Hành/Visualize Data/Preprocessed Data/entities/course_eda.json')

id,name,prerequisites,about,resource,name_trans,about_trans,about_segmented,name_segmented,field
str,str,str,str,list[struct[3]],str,str,str,str,list[str]
"""C_584313""","""《资治通鉴》导读""","""""","""通过老师导读，同学们可深入这…","[{[""第一课 导论与三家分晋"", ""导论"", ""导论""],""V_849"",""1.1.1""}, {[""第一课 导论与三家分晋"", ""智伯的覆亡"", ""智伯的覆亡""],""V_850"",""1.2.1""}, … {[""第十五课 隋唐霸业"", null, ""第十五课 隋唐霸业--习题""],""Ex_957"",""15.8""}]","""giới thiệu về …","""thông qua sự h…","""thông_qua sự h…","""giới_thiệu về …","[""lịch sử"", ""trường dạy tiếng trung""]"
"""C_584329""","""微积分——极限理论与一元函数…","""""","""本课程是理工科的一门数学基础…","[{[""序言"", ""序言"", ""序言""],""V_1350"",""1.1.1""}, {[""第一章 实数与函数"", ""第一节 实数集的界与确界"", ""实数集的界""],""V_1351"",""2.1.1""}, … {[""第八章 级数"", null, ""第八章 级数--第六节思考与练习""],""Ex_1545"",""9.9""}]","""giải tích - lý…","""khóa học này l…","""khoá học này l…","""giải_tích - lý…","[""toán học"", ""kinh tế học ứng dụng"", … ""kinh tế lý thuyết""]"
"""C_584381""","""新闻摄影""","""""","""掌握基本的摄影技能，了解图片…","[{[""第一章 绪论"", ""第一讲 引言1"", ""引言1""],""V_1800"",""1.1.1""}, {[""第一章 绪论"", ""第二讲 引言2"", ""引言2""],""V_1801"",""1.2.1""}, … {[""大作业提交"", null, ""《大作业》提交--小节""],""Ex_1926"",""20.4""}]","""chụp ảnh tin t…","""nắm vững các k…","""nắm vững các k…","""chụp ảnh tin_t…","[""báo chí"", ""nghệ thuật""]"
"""C_597208""","""数据挖掘：理论与算法""","""""","""最有趣的理论+最有用的算法=…","[{[""走进数据科学：博大精深，美不胜收"", ""整装待发"", ""Video""],""V_2961"",""1.1.1""}, {[""走进数据科学：博大精深，美不胜收"", ""学而不思则罔"", ""Video""],""V_2962"",""1.3.1""}, … {[""美丽数据说：阆苑仙葩，美玉无瑕"", null, ""第十一章第一节测试题""],""Ex_3104"",""11.1""}]","""khai thác dữ l…","""lý thuyết thú …","""lý_thuyết thú_…","""khai_thác dữ_l…","[""khoa học và công nghệ máy tính""]"
"""C_597225""","""大学计算机""","""""","""大学计算机课程将以计算思维为…","[{[""第1周： 基于计算机的问题求解"", ""课程介绍"", ""开篇""],""V_4596"",""1.1.1""}, {[""第1周： 基于计算机的问题求解"", ""1.0 本章导学"", ""1.0 本章导学""],""V_4597"",""1.2.1""}, … {[""第9周：算法与程序设计"", null, ""第九周测验""],""Ex_4827"",""10.12""}]","""máy tính đại h…","""các khóa học m…","""các khoá học m…","""máy_tính đại_h…",null
"""C_597229""","""财务分析与决策""","""""","""这门课程用财务语言解构企业的…","[{[""资金的运用——认识资产"", ""1.1 绪论"", ""绪论""],""V_5042"",""2.1.1""}, {[""资金的运用——认识资产"", ""1.2 认识资产负债表"", ""认识资产负债表""],""V_5043"",""2.2.1""}, … {[""期末大作业——主观题"", null, ""期末主观题""],""Ex_5144"",""12.2""}]","""phân tích tài …","""khóa học này s…","""khoá học này s…","""phân_tích tài_…","[""kinh tế học ứng dụng"", ""khoa học và kỹ thuật quản lý""]"
"""C_597291""","""高级英语写作""","""""","""本课程能够帮助学生掌握英语段…","[{[""Chapter One Paragraph Writing"", ""1.1 Parts of a Paragraph"", ""1.1 Parts of a Paragraph""],""V_8379"",""1.1.1""}, {[""Chapter One Paragraph Writing"", ""1.2 Four Steps in Writing"", ""1.2.1 Step 1- Begin with a point""],""V_8380"",""1.2.1""}, … {[""Chapter Five Research Paper Writing"", null, ""Chapter Five Research Paper Writing""],""Ex_8432"",""5.3""}]","""viết tiếng anh…","""khóa học này c…","""khoá học này c…","""viết tiếng anh…",null
"""C_597307""","""大唐兴衰""","""""","""隋唐五代史是史学名著《资治通…","[{[""第一课、隋朝开基"", ""第一节 隋帝杨坚"", ""第一节 隋帝杨坚""],""V_9394"",""1.1.1""}, {[""第一课、隋朝开基"", ""第二节 杨隋代周"", ""第二节 杨隋代周""],""V_9395"",""1.2.1""}, … {[""第十二课、落日长安"", null, ""第十二课、落日长安--习题""],""Ex_9464"",""12.5""}]","""sự thăng trầm …","""lịch sử nhà tù…","""lịch_sử nhà tu…","""sự thăng_trầm …","[""lịch sử""]"
"""C_597365""","""五分钟轻松搞定职场礼仪（20…","""""","""职场“礼”为先，成功的未来不…","[{[""课程介绍动画：职场“礼”为先，成功的未来不是梦"", ""课程介绍动画"", ""课程介绍动画 — 职场“礼”为先，成功的未来不是梦""],""V_15713"",""1.1.1""}, {[""第一章 塑造优雅、大方、可亲的职业形象"", ""第一节 坐姿篇"", ""本讲导学 坐姿""],""V_15714"",""2.1.1""}, … {[""第五章 应对繁杂、多变、细致的文化差异"", ""第四节 欧美篇"", ""授课视频 讲义 — 欧美各国礼仪""],""V_15800"",""6.4.1""}]","""dễ dàng nắm vữ…","""“nghi thức” đư…","""“ nghi_thức ” …","""dễ_dàng nắm vữ…",null


In [6]:
class DataLoader:
    def __init__(self):
        pass

    def load_course_feats_1(self, course_path, c_map_path, u_map_path):
        # Load mapping
        org2idx_user, idx2org_user, org2idx_course, idx2org_course = self.load_mapping(c_map_path, u_map_path)

        # Load course
        feat_cols = ['about_segmented', 'name_segmented', 'field']
        feat_types = ['tfidf', 'tfidf', 'tfidf'] # Change if necessary
        course_df = pl.read_ndjson(course_path) \
                        .select(['id'] + feat_cols) \
                        .filter(pl.col('id').is_in(idx2org_course)) \
                        .with_columns(
                            pl.col('id').map_dict(org2idx_user).cast(pl.Int64),
                            pl.col('field').fill_null([])) \
                        .sort('id')

        # Create feature
        feat_dict = {}

        for col, type_ in zip(feat_cols, feat_types):
            enc, feats = self.get_feats_for_col(course_df, col, type_)
            feat_dict[col] = (enc, feats)
            print(f'{col} - {type_} - feature shape: {feats.shape}')

        del feat_cols, feat_types, org2idx_user, idx2org_user, org2idx_course, idx2org_course
        gc.collect()

        return feat_dict

    def load_course_feats_2(self, kg_path, c_map_path, u_map_path):
        # Load mapping
        org2idx_user, idx2org_user, org2idx_course, idx2org_course = self.load_mapping(c_map_path, u_map_path)

        # Load course
        feat_cols = ['concept', 'school']
        feat_types = ['one_hot', 'one_hot'] # Change if necessary
        kg_data = pl.read_csv(kg_path, separator=' ', has_header=False, new_columns=['id', 'r', 't'])
        concept_df = kg_data.filter(pl.col('r') == 0) \
                            .group_by('id') \
                            .agg(pl.col('t')) \
                            .rename({'t': 'concept'})

        school_df = kg_data.filter(pl.col('r') == 1) \
                            .group_by('id') \
                            .agg(pl.col('t')) \
                            .rename({'t': 'school'})

        course_df = pl.from_dict({'id': range(len(idx2org_course))}) \
                                .join(concept_df, on='id', how='left') \
                                .join(school_df, on='id', how='left') \
                                .with_columns(pl.all().exclude('id').fill_null([]))
        del concept_df, school_df; gc.collect()

        # Create feature
        feat_dict = {}

        for col, type_ in zip(feat_cols, feat_types):
            enc, feats = self.get_feats_for_col(course_df, col, type_)
            feat_dict[col] = (enc, feats)
            print(f'{col} - {type_} - feature shape: {feats.shape}')

        del feat_cols, feat_types, org2idx_user, idx2org_user, org2idx_course, idx2org_course
        gc.collect()

        return feat_dict

    def get_feats_for_col(self, df, col, t):
        enc = None

        if t == 'one_hot':
            enc = MultiLabelBinarizer()
        elif t == 'tfidf':
            if df[col].dtype == pl.List:
                enc = Pipeline([
                    ('cv', CountVectorizer(analyzer=lambda x: x)),
                    ('tfidf', TfidfTransformer())
                ])
            else:
                enc = TfidfVectorizer()
        else:
            raise f"{col} must be in ('one_hot', 'tfidf)"

        feats = csr_matrix(enc.fit_transform(df[col]))

        return enc, feats


    def load_mapping(self, c_map_path, u_map_path):
        user_df = pl.read_csv(u_map_path, separator=' ')
        org2idx_user = dict(zip(user_df['org_id'], user_df['remap_id']))
        idx2org_user = user_df['org_id'].to_list()

        course_df = pl.read_csv(c_map_path,  separator=' ')
        org2idx_course = dict(zip(course_df['org_id'], course_df['remap_id']))
        idx2org_course = course_df['org_id'].to_list()

        return org2idx_user, idx2org_user, org2idx_course, idx2org_course

    def load_user_dict(self, inter_path):
        user_dict = {}

        with open(inter_path) as rf:
            for line in rf:
                line = list(map(int, line.strip().split()))
                user, courses = line[0], line[1:]
                user_dict[user] = courses

        return user_dict

In [7]:
data_loader = DataLoader()
train_user_dict = data_loader.load_user_dict(os.path.join(MODEL_DATA_DIR, 'train.txt'))
test_user_dict = data_loader.load_user_dict(os.path.join(MODEL_DATA_DIR, 'test.txt'))
courses_feats = data_loader.load_course_feats_2(
    os.path.join(MODEL_DATA_DIR, 'kg_final.txt'),
    os.path.join(MODEL_DATA_DIR, 'item_list.txt'),
    os.path.join(MODEL_DATA_DIR, 'user_list.txt')
)

concept - one_hot - feature shape: (2831, 7162)
school - one_hot - feature shape: (2831, 144)


# 2. Build model

In [9]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score, log_loss, mean_squared_error

def precision_at_k_batch(hits, k):
    """
    calculate Precision@k
    hits: array, element is binary (0 / 1), 2-dim
    """
    res = hits[:, :k].mean(axis=1)
    return res

def ndcg_at_k_batch(hits, k):
    """
    calculate NDCG@k
    hits: array, element is binary (0 / 1), 2-dim
    """
    hits_k = hits[:, :k]
    dcg = np.sum((2 ** hits_k - 1) / np.log2(np.arange(2, k + 2)), axis=1)

    sorted_hits_k = np.flip(np.sort(hits), axis=1)[:, :k]
    idcg = np.sum((2 ** sorted_hits_k - 1) / np.log2(np.arange(2, k + 2)), axis=1)

    idcg[idcg == 0] = np.inf
    ndcg = (dcg / idcg)
    return ndcg

def recall_at_k_batch(hits, k):
    """
    calculate Recall@k
    hits: array, element is binary (0 / 1), 2-dim
    """
    res = (hits[:, :k].sum(axis=1) / hits.sum(axis=1))
    return res

def calc_metrics_at_k(cf_scores, train_user_dict, test_user_dict, user_ids, item_ids, Ks, num_negatives=100):
    '''
    Calculate precision, recall, and NDCG at K for each user with negative sampling.
    Negative sampling selects 100 items that are neither the ground truth item nor in the user's train set.

    cf_scores: (n_users, n_items)
    '''

    binary_hit = []
    temp_cf_scores = []
    test_indices = []

    test_pos_item_binary = np.concatenate((
        np.ones((len(user_ids), 1)),
        np.zeros((len(user_ids), num_negatives))
    ), axis=1)

    for idx, user in enumerate(user_ids):
        # Ground truth items for the user
        test_item = set(test_user_dict[user])

        # Items in the training set to be excluded
        train_items = set(train_user_dict[user])

        # Negative samples: items not in the test items and not in the train items
        possible_negatives = [item for item in item_ids if item not in train_items and item not in test_item]
        negative_samples = np.random.choice(possible_negatives, num_negatives, replace=False)

        # Selected items for testing: ground truth + negative samples
        test_set = list(test_item) + list(negative_samples)
        # test_indices.append(test_set)

        # Get the corresponding scores of these items from the cf_scores matrix
        temp_cf_scores.append(cf_scores[idx][test_set].tolist())

    try:
        _, rank_indices = torch.sort(torch.LongTensor(temp_cf_scores).cuda(), descending=True)    # try to speed up the sorting process
    except:
        _, rank_indices = torch.sort(torch.LongTensor(temp_cf_scores), descending=True)

    rank_indices = rank_indices.cpu()

    # binary_hit = [] # shape (n_users, num_negatives+1)
    # test_indices = np.asarray(test_indices)

    for i in range(len(user_ids)):
        binary_hit.append(test_pos_item_binary[i][rank_indices[i]])
    binary_hit = np.array(binary_hit, dtype=np.float32)

    metrics_dict = {}
    for k in Ks:
        metrics_dict[k] = {}
        metrics_dict[k]['precision'] = precision_at_k_batch(binary_hit, k)
        metrics_dict[k]['recall']    = recall_at_k_batch(binary_hit, k)
        metrics_dict[k]['ndcg']      = ndcg_at_k_batch(binary_hit, k)

    return metrics_dict

In [10]:
class CBF:
    def __init__(self):
        self.course_feats = None
        self.train_user_dict = None

    def fit(self, courses_feats, train_user_dict):
        self.courses_feats = courses_feats
        self.train_user_dict = train_user_dict
        self.n_users = len(train_user_dict)
        for col, (enc, feats) in self.courses_feats.items():
            self.n_items = feats.shape[0]
            break

    def predict(self, user_batch, weights=None):
        # Get features of users
        users_feats = {col: [] for col in self.courses_feats.keys()}

        for user in user_batch:
            used_courses = self.train_user_dict[user]
            for col, (enc, courses_feat) in self.courses_feats.items():
                users_feats[col].append(courses_feat[used_courses].mean(axis=0))

        # Calculate cf_scores
        if weights:
            assert len(weights) == len(self.courses_feats), 'weights must correspond to feat types'
        else:
            weights = [1] * len(self.courses_feats)

        for idx, col in enumerate(users_feats):
            users_feat = np.asarray(np.row_stack(users_feats[col]))
            courses_feat = self.courses_feats[col][1]
            weight = weights[idx]
            if idx == 0:
                cf_scores = weight * cosine_similarity(users_feat, courses_feat)
            else:
                cf_scores = cf_scores + weight * cosine_similarity(users_feat, courses_feat)

        return cf_scores

    def recommend(self, user_batch, topk=10, weights=None):
        cf_scores = self.predict(user_batch, weights)

        for idx, user in enumerate(user_batch):
            cf_scores[idx][self.train_user_dict[user]] = -10

        ranked_courses = np.argsort(cf_scores)[:, ::-1]
        return ranked_courses[:topk]

    def eval(self, test_user_dict, Ks, batch_size=500, weights=None):
        # Predict
        user_batches = list(range(len(test_user_dict)))
        user_batches = [user_batches[i: i+batch_size] for i in range(0, len(user_batches), batch_size)]

        metric_names = ['precision', 'recall', 'ndcg']
        metrics_dict = {k: {m: 0 for m in metric_names} for k in Ks}

        item_ids = list(range(self.n_items))

        with tqdm(total=len(user_batches), desc='Evaluating Iteration') as pbar:
            for user_batch in user_batches:
                batch_cf_scores = self.predict(user_batch)
                batch_metrics = calc_metrics_at_k(
                    batch_cf_scores, self.train_user_dict, test_user_dict,
                    user_batch, item_ids, Ks, num_negatives=100
                )

                for k in Ks:
                    for m in metric_names:
                        metrics_dict[k][m] += batch_metrics[k][m].sum()

                pbar.update(1)

        # Cal mean
        for k in Ks:
            print(f'====== @{k} ======')
            for m in metric_names:
                metrics_dict[k][m] /= len(test_user_dict)
                print(f'{m}     : {metrics_dict[k][m]}')

        return metrics_dict

# 3. Predict

In [11]:
cf = CBF()
cf.fit(courses_feats, train_user_dict)
cf.recommend([0, 1])

array([[1019, 1027, 1033, ...,  614,  216, 1538],
       [1534,  803, 2098, ..., 1143, 1725,  834]])

# 4. Evaluation

In [13]:
cf.eval(test_user_dict, Ks=[1, 5, 10])

Evaluating Iteration: 100%|██████████| 200/200 [03:17<00:00,  1.01it/s]

====== @1 ======
precision     : 0.07090197961368024
recall     : 0.07090197961368024
ndcg     : 0.07090197961368024
====== @5 ======
precision     : 0.01789154667296189
recall     : 0.08945773189688803
ndcg     : 0.08067587664719021
====== @10 ======
precision     : 0.010588282586350814
recall     : 0.10588282367533935
ndcg     : 0.08571630359438237


{1: {'precision': 0.07090197961368024,
  'recall': 0.07090197961368024,
  'ndcg': 0.07090197961368024},
 5: {'precision': 0.01789154667296189,
  'recall': 0.08945773189688803,
  'ndcg': 0.08067587664719021},
 10: {'precision': 0.010588282586350814,
  'recall': 0.10588282367533935,
  'ndcg': 0.08571630359438237}}